## Face detection
Detect, classify and localize face using Deep learning and CNN

### Import libraries and define vars

In [44]:
# import all libraries
import tensorflow as tf
import keras
from keras.layers import *
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import json
import numpy as np
import cv2
import os
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sb

In [26]:
EPOCHS = 100
BATCH_SIZE = 16
DATA_PATH = os.path.join("data")

### Load data functions

In [27]:
def load_images(image):
    """
    This function will load and decode images
    """
    # read and decode
    load = tf.io.read_file(image)
    decoded = tf.io.decode_jpeg(load)
    
    return decoded

In [28]:
def load_labels(label_path):
    """
    This function will load labels and return class and bboxes
    """
    # read the label file (.json format)
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    # get the bounding box coordinates and class
    label_class = [label['class']]
    label_bbox = label['bbox']
    
    return label_class, label_bbox

### Create dataset 

 We have to load our data (images and labels), then create our final dataset using `tensorflow`

1. Load images 

In [29]:
# load train images
train_images = tf.data.Dataset.list_files('data/train/images/*.jpg', shuffle=False)
train_images = train_images.map(load_images)
train_images = train_images.map(lambda x: tf.image.resize(x, (120,120)))
train_images = train_images.map(lambda x: x/255)

# load validation images
val_images = tf.data.Dataset.list_files('data/val/images/*.jpg', shuffle=False)
val_images = val_images.map(load_images)
val_images = val_images.map(lambda x: tf.image.resize(x, (120,120)))
val_images = val_images.map(lambda x: x/255)

2. Load labels

In [30]:
# load train labels
train_labels = tf.data.Dataset.list_files('data/train/labels/*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

# load validation labels
val_labels = tf.data.Dataset.list_files('data/val/labels/*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(load_labels, [x], [tf.uint8, tf.float16]))

3. Create final dataset 

In [31]:
# create train dataset
train_dataset = tf.data.Dataset.zip((train_images, train_labels))
train_dataset = train_dataset.shuffle(5000)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(4)

In [32]:
# create validation dataset
val_dataset = tf.data.Dataset.zip((val_images, val_labels))
val_dataset = val_dataset.shuffle(5000)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(4)

In [33]:
train_dataset.as_numpy_iterator().next()[0].shape

(16, 120, 120, 3)

### Neural Networks

To create the neural networks we are using VGG-16 as our base model 

In [34]:
def build_model():
    # input layer
    input_layer = Input(shape=(120, 120, 3))
    
    # vgg16 model
    vgg = VGG16(include_top=False)(input_layer)
    
    # layers for classification
    x = GlobalAveragePooling2D()(vgg)
    x = Dense(128)(x)
    x = Dense(128)(x)
    x = Dropout(0.2)(x)
    x = Dense(1, activation="sigmoid")(x)
    
    # layers for detection
    y = GlobalAveragePooling2D()(vgg)
    y = Dense(2048, activation="relu")(y)
    y = Dense(2048, activation="relu")(y)
    y = Dropout(0.4)(y)
    y = Dense(4, activation="sigmoid")(y)

    # create the final model
    model = Model(inputs=input_layer, outputs=[x, y])

    return model

In [35]:
# define the detection model
detection_model = build_model()
detection_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg16 (Functional)             (None, None, None,   14714688    ['input_3[0][0]']                
                                512)                                                              
                                                                                                  
 global_average_pooling2d_2 (Gl  (None, 512)         0           ['vgg16[0][0]']                  
 obalAveragePooling2D)                                                                      

### Train the model

In [36]:
def localization_loss(y_true, yhat):
    """
    This function is the loss function of object localization
    """
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2]))
                  
    h_true = y_true[:,3] - y_true[:,1] 
    w_true = y_true[:,2] - y_true[:,0] 

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    
    return delta_coord + delta_size

In [37]:
# define losses
classification_loss = BinaryCrossentropy()
detection_loss = localization_loss

# define optimizer
batches_per_epoch = len(train_dataset)
learning_rate_decay = (1./0.75 - 1) / batches_per_epoch
optimizer = Adam(learning_rate=0.0001, decay=learning_rate_decay)

Custom training loop: 

In [38]:
class FaceDetectionModel(Model):
    def __init__(self, trackermodel, **kwargs):
        super().__init__(**kwargs)
        self.model = trackermodel

    def compile(self, optimizer, classification_loss, detection_loss, **kwargs):
        """
        This function will compile the model
        """
        super().compile(**kwargs)
        # define losses and optimizer
        self.classification_loss = classification_loss
        self.detection_loss = detection_loss
        self.optimizer = optimizer
    
    def train_step(self, batch, **kwargs):
        """
        This function containes a train step of our model
        """
        # define input and output datas as X and y
        X, y = batch

        # backpropegation
        with tf.GradientTape() as tape:
            # make model to train on classes and coordinates
            classes, coords = self.model(X, training=True)
            # calculate losses
            class_loss = self.classification_loss(y[0], classes)
            loc_loss = self.detection_loss(tf.cast(y[1], tf.float32), coords)

            # calculate the total loss
            total_loss = loc_loss  +0.5 * class_loss

            # define the gradient
            grad = tape.gradient(total_loss, self.model.trainable_variables)

        # apply gradient
        optimizer.apply_gradients(zip(grad, self.model.trainable_variables))

        return {"total_loss":total_loss, "class_loss":class_loss, "loc_loss":loc_loss}

    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)


In [39]:
# define the final model
model = FaceDetectionModel(detection_model)

In [40]:
# compile the detection model
model.compile(optimizer, classification_loss, detection_loss)

In [41]:
# train model
history = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset)

Epoch 1/100
494/494 [==============================] - 57s 101ms/step - total_loss: 0.4331 - class_loss: 0.0557 - loc_loss: 0.4052 - val_loss: 0.0000e+00
Epoch 2/100
494/494 [==============================] - 50s 92ms/step - total_loss: 0.0847 - class_loss: 0.0054 - loc_loss: 0.0821 - val_loss: 0.0000e+00
Epoch 3/100
494/494 [==============================] - 50s 93ms/step - total_loss: 0.0603 - class_loss: 0.0038 - loc_loss: 0.0584 - val_loss: 0.0000e+00
Epoch 4/100
494/494 [==============================] - 50s 93ms/step - total_loss: 0.0464 - class_loss: 0.0023 - loc_loss: 0.0452 - val_loss: 0.0000e+00
Epoch 5/100
494/494 [==============================] - 50s 93ms/step - total_loss: 0.0410 - class_loss: 0.0013 - loc_loss: 0.0404 - val_loss: 0.0000e+00
Epoch 6/100
494/494 [==============================] - 51s 94ms/step - total_loss: 0.0332 - class_loss: 8.3962e-04 - loc_loss: 0.0328 - val_loss: 0.0000e+00
Epoch 7/100
494/494 [==============================] - 51s 94ms/step - total_

In [52]:
# save the model
model.save_weights("model.h5")

### Use the model

In [54]:
model.load_weights('model.h5')

In [56]:
# get the webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[50:500, 50:500, :]

    # ready our frames for detection
    rgb_images = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized_img = tf.image.resize(rgb_images, (120, 120))

    # make predictions
    prediction = model.predict(np.expand_dims(resized_img/255,0))
    # get the coordinates
    coords = prediction[1][0]

    # draw square if prediction accuracy was high
    if prediction[0] >= 0.85:
        cv2.rectangle(frame, 
                      tuple(np.multiply(coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 1)

    # show the frame 
    cv2.imshow("Face detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 16ms/step
